# First steps for Google Places API

**To Do**

- seit wann Gemini Modell

Meine Aufgaben
- Datensatz vorbereiten: Tübingen
- zwei Datensätze: Name und Place_id + alle weiteren infos


## Preparations

In [ ]:
# install all packages
# !pip install python-dotenv
# !pip install googlemaps
# !pip install requests

In [ ]:
### apperaently this is not needed
#!pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
# execute the quickstart.py file to authenticate the user
# !python quickstart.py

In [1]:
import requests
from urllib.parse import urlencode
import googlemaps
import os
from dotenv import load_dotenv
import time
import pandas as pd

In [5]:
# import the api_key from the api_key.py file
from api_key import api_key

## keyword search 
**read** https://developers.google.com/maps/documentation/places/web-service/search-nearby

can further include
- maxprice / minprice
- opennow

### keyword + location /nearbysearch

#### cleaning function

In [2]:
def tidy_nearby_result(info):
    # Define a function to tidy a single result
    def tidy_get_result(result):
        return {
            "name": result.get("name"),
            "place_id": result.get("place_id"),
            "address": result.get("vicinity"),
            "latitude": result.get("geometry", {}).get("location").get("lat"),
            "longitude": result.get("geometry", {}).get("location").get("lng"),
            "rating": result.get("rating"),
            "user_ratings_total": result.get("user_ratings_total"),
            "price_level": result.get("price_level"),
            "types": result.get("types"),
            "opening_hours": result.get("opening_hours"),
            #"opening_hours": result.get("opening_hours", {}).get("open_now"),
            "business_status": result.get("business_status"),
        }

    # Apply tidy_result to each entry in info["results"] and create a list of dictionaries
    tidy_results_list = [tidy_get_result(result) for result in info.get("results", [])]
    
    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(tidy_results_list)
    return df

#### get place location /findplacefromtext

In [11]:
def get_place_info(address):
# Base URL
  base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"
# Parameters in a dictionary
  params = {
   "input": address,
   "inputtype": "textquery",
   "fields": "name,geometry/location,place_id",
   "key": api_key,
  }
# Send request and capture response
  response = requests.get(base_url, params=params)
# Check if the request was successful
  if response.status_code == 200:
    return response.json()
  else:
    return None

In [20]:
info = get_place_info("ingolstadt")	
print(info)

longitude =  info["candidates"][0].get("geometry", {}).get("location", {}).get("lng")
latitude = info["candidates"][0].get("geometry", {}).get("location", {}).get("lat")


{'candidates': [{'geometry': {'location': {'lat': 48.7667395, 'lng': 11.4226498}}, 'name': 'Ingolstadt', 'place_id': 'ChIJuwo2iFH-nkcR6op0EyJ-Xew'}], 'status': 'OK'}


#### 1.1 get 20 results - default

In [3]:
def nearby_search(keyword, lat, lng):
    # Base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Parameters in a dictionary
    params = {
        "key": api_key,
        "location": f"{lat},{lng}", # must be specified as latitude,longitude
        "radius": 1500, # or rankby=distance
        "keyword": keyword, # can be the name of the restaurant or the type of restaurant
        # "field": "...", # does not work for nearby search
        "language": "en",
        "type": "restaurant" # not clear if it works
    }
    # Send request and capture response
    response = requests.get(base_url, params=params)
    # Check if the request was successful
    if response.status_code == 200:
         # Pass the JSON response data to the tidy function
        info = response.json()
        return tidy_nearby_result(info)
    else:
          print("Error:", response.status_code)

In [25]:
info = nearby_search("westpark", latitude, longitude)
info.head()

,name,place_id,address,latitude,longitude,rating,user_ratings_total,price_level,types,opening_hours,business_status
0,Mooshäusl,ChIJWQWSa2H-nkcRfbj3PnReCyQ,"Mooshäuslweg, Ingolstadt",48.762589,11.401645,4.4,1045,2.0,"[restaurant, food, bar, point_of_interest, est...",{'open_now': True},OPERATIONAL
1,Cocoon Restaurant Ingolstadt,ChIJGQPfW8__nkcRTLOgxyKLle8,"Neuburger Str. 19, Ingolstadt",48.768769,11.408663,4.4,834,2.0,"[restaurant, food, point_of_interest, establis...",{'open_now': False},OPERATIONAL
2,Weissbräuhaus zum Herrnbräu,ChIJuTcbPVH-nkcR1bE-NTwKBqY,"Dollstraße 3, Ingolstadt",48.763372,11.423738,4.3,2341,2.0,"[restaurant, food, point_of_interest, establis...",{'open_now': True},OPERATIONAL
3,"das ""Mo"" - Neue Galerie",ChIJKbSBa1r-nkcRTVJ3FetRS0M,"Bergbräustraße 7, Ingolstadt",48.764304,11.418922,4.4,2426,2.0,"[restaurant, food, bar, point_of_interest, est...",{'open_now': False},OPERATIONAL
4,Amori Ingolstadt Café Trattoria Bar,ChIJKTdnnF3-nkcR2Bv5nOpaeVQ,"Gerolfinger Str. 15, Ingolstadt",48.766117,11.408617,4.4,231,2.0,"[restaurant, cafe, food, store, point_of_inter...",{'open_now': True},OPERATIONAL


#### 1.2 60 results - extension

In [ ]:
def nearby_search_all_results(keyword, lat, lng):
    # Base URL
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # Initial parameters without `pagetoken`
    params = {
        "key": api_key,
        "location": f"{lat},{lng}",  # Latitude and longitude
        "radius": 1500,  # Search radius in meters
        "keyword": keyword,  # Keyword for search
        "type": "restaurant",  # Restrict to restaurants
        "language": "en"
    }
    
    all_results = []  # To store all results across pages

    while True:
        # Send request and capture response
        response = requests.get(base_url, params=params)
        
        # Check if the request was successful
        if response.status_code == 200:
            data = response.json()
            all_results.extend(data.get("results", []))  # Append current page results to list

            # Check if there's a next_page_token for additional results
            next_page_token = data.get("next_page_token")
            if next_page_token:
                # Update params with pagetoken for the next request
                params["pagetoken"] = next_page_token

                # Wait at least 2 seconds before making the next request
                time.sleep(2) # needed to avoid OVER_QUERY_LIMIT
            else:
                break  # No more pages, exit loop
        else:
            print("Error:", response.status_code)
            break

    # Use tidy_nearby_result to format the results as a DataFrame
    return tidy_nearby_result({"results": all_results})

In [39]:
# Example usage
place_info = nearby_search_all_results("restaurant", latitude, longitude)
place_info.head()

,name,place_id,address,latitude,longitude,rating,user_ratings_total,price_level,types,opening_hours,business_status
0,The little Kitchen,ChIJg0eJ3XD_nkcRdoN56jjk6gQ,"Am Viktualienmarkt 6, Ingolstadt",48.762952,11.426687,4.9,26,NaN,"[restaurant, food, point_of_interest, establis...",{'open_now': True},OPERATIONAL
1,Gasthaus Daniel,ChIJ4zhE2lD-nkcR84vDBWMbvZQ,"Roseneckstraße 1, Ingolstadt",48.763165,11.422513,4.5,1878,2.0,"[restaurant, food, point_of_interest, establis...",{'open_now': True},OPERATIONAL
2,Weissbräuhaus zum Herrnbräu,ChIJuTcbPVH-nkcR1bE-NTwKBqY,"Dollstraße 3, Ingolstadt",48.763372,11.423738,4.3,2341,2.0,"[restaurant, food, point_of_interest, establis...",{'open_now': True},OPERATIONAL
3,Mooshäusl,ChIJWQWSa2H-nkcRfbj3PnReCyQ,"Mooshäuslweg, Ingolstadt",48.762589,11.401645,4.4,1045,2.0,"[restaurant, food, bar, point_of_interest, est...",{'open_now': True},OPERATIONAL
4,Sultanahmet Köftecisi Ingolstadt,ChIJ5Yndj4r_nkcRZ8vZl6qSuVg,"Friedrich-Ebert-Straße 37, Ingolstadt",48.771954,11.441074,4.3,136,NaN,"[restaurant, food, point_of_interest, establis...",{'open_now': True},OPERATIONAL


### only keywords & one result /findplacefromtext
- best to find a single, highly relevant place

Documentation: https://developers.google.com/maps/documentation/places/web-service/search-find-place 
Caution: Place Search requests and Place Details requests do not return the same fields. Place Search requests return a subset of the fields that are returned by Place Details requests. If the field you want is not returned by Place Search, you can use Place Search to get a place_id, then use that Place ID to make a Place Details request.

fields possibilities: https://developers.google.com/maps/documentation/places/web-service/place-data-fields#places-api-fields-support
interesting for our use case: 
    name,place_id,type,url,formatted_address,user_ratings_total,rating,price_level

maybe interesting:
    address_components,adr_address,business_status,geometry,geometry/viewpoint,geometry/location,photo,plus_code,utc_offset,vicinity,wheelchair_accessible_entrance,current_opening_hours,formatted_phone_number,international_phone_number,opening_hours,secondary_opening_hours,website, curbside_pickup,delivery, dine_in,editorial_summary,reservable,serves_beer,serves_breakfast,serves_brunch,serves_dinner, serves_lunch,serves_vegetarian_food,serves_wine,takeout

In [ ]:
def find_place_from_text(input):
    # Base URL
    base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json?"
    # Parameters in a dictionary
    params = {
        "input": input,  # Text string for search, such as a place name or address
        "inputtype": "textquery",
        "fields": "name,place_id,type,formatted_address,user_ratings_total,rating,price_level,geometry/location", 
        "key": api_key,
        "language": "en",
        "type": "restaurant"
    }
    # Send request and capture response
    response = requests.get(base_url, params=params)
    # Check if the request was successful
    if response.status_code == 200:
         return response.json()
    else:
          print("Error:", response.status_code)

In [43]:
info = find_place_from_text("falafel ingolstadt")
info

{'candidates': [{'formatted_address': 'Oberer Graben 73, 85049 Ingolstadt, Germany',
   'geometry': {'location': {'lat': 48.7678443, 'lng': 11.4220422}},
   'name': 'Restaurant Grill Oriental',
   'place_id': 'ChIJoQyDMJr_nkcRg823kSW_5Kg',
   'rating': 4.2,
   'types': ['restaurant', 'food', 'point_of_interest', 'establishment'],
   'user_ratings_total': 314}],
 'status': 'OK'}

## 3. get details /place/details
https://developers.google.com/maps/documentation/places/web-service/place-details 

Interesting:
accessibilityOptions,businessStatus,types,googleMapsUri,primaryType, primaryTypeDisplayName, currentOpeningHours, currentSecondaryOpeningHours, internationalPhoneNumber, nationalPhoneNumber, priceLevel, priceRange, rating, regularOpeningHours, regularSecondaryOpeningHours, userRatingCount, websiteUri

Maybe also interesting:
allowsDogs, curbsidePickup, delivery, dineIn, editorialSummary, evChargeOptions, fuelOptions, goodForChildren, goodForGroups, goodForWatchingSports, liveMusic, menuForChildren, parkingOptions, paymentOptions, outdoorSeating, reservable, restroom, reviews, routingSummaries,* servesBeer, servesBreakfast, servesBrunch, servesCocktails, servesCoffee, servesDessert, servesDinner, servesLunch, servesVegetarianFood, servesWine, takeout


for later:  Generate Google Maps link
f"https://www.google.com/maps/place/?q=place_id:{place_id}"

In [ ]:
def get_place_details(place_id):
    # Base URL
    base_url = f"https://places.googleapis.com/v1/places/{place_id}"
    # Parameters in a dictionary
    params = {
        "fields": "displayName,formattedAddress,rating,userRatingsTotal,geometry/location,businessStatus,openingHours/openNow",
        "key": api_key,
        "language": "en"
    }
    # Send request and capture response
    response = requests.get(base_url, params=params)
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print("Error:", response.status_code)

In [56]:
details = get_place_details("ChIJ_wgzYiHgnUcR_9WTTO4iT88")
details

Error: 403


In [51]:
def get_place_details(place_id):
    # Base URL
    base_url = "https://maps.googleapis.com/maps/api/place/details/json"
    # Parameters in a dictionary
    params = {
        "place_id": place_id,
        "fields": "name,formatted_address,formatted_phone_number,website,price_level,rating,user_ratings_total,geometry/location,pureServiceAreaBusiness",
        "key": api_key,
        "language": "en"
    }
    # Send request and capture response
    response = requests.get(base_url, params=params)
    # Check if the request was successful
    if response.status_code == 200:
         return response.json()
    else:
          print("Error:", response.status_code)

In [ ]:
def tidy_nearby_result(info):
    # Define a function to tidy a single result
    def tidy_get_result(result):
        geometry = result.get("geometry", {})
        location = geometry.get("location", {})
        
        return {
            "name": result.get("displayName"),  # Updated field name
            "place_id": result.get("id"),  # Updated field name
            "address": result.get("formattedAddress"),  # Updated field name
            "latitude": location.get("lat"),
            "longitude": location.get("lng"),
            "rating": result.get("rating"),
            "user_ratings_total": result.get("userRatingsTotal"),
            "price_level": result.get("priceLevel"),
            "types": result.get("types"),
            "opening_hours": result.get("openingHours", {}).get("openNow"),
            "business_status": result.get("businessStatus"),
            "google_maps_link": f"https://www.google.com/maps/place/?q=place_id:{result.get('id')}"  # Manually constructed link
        }

    # Apply tidy_result to each entry in info["results"] and create a list of dictionaries
    tidy_results_list = [tidy_get_result(result) for result in info.get("results", [])]
    
    # Convert the list of dictionaries to a pandas DataFrame
    df = pd.DataFrame(tidy_results_list)
    return df

def get_place_details(place_id):
    # Base URL for the new Place Details API
    base_url = f"https://places.googleapis.com/v1/places/{place_id}"
    
    # Parameters in a dictionary
    params = {
        "fields": "displayName,formattedAddress,formattedPhoneNumber,websiteUri,priceLevel,rating,userRatingsTotal,geometry/location,businessStatus,openingHours/openNow", 
        "key": api_key,
        "language": "en"
    }
    
    # Send request and capture response
    response = requests.get(base_url, params=params)
    
    # Check if the request was successful
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Error: {response.status_code} - {response.text}")
        return None

# Example usage
place_id = "ChIJj61dQgK6j4AR4GeTYWZsKWw"  # Replace with your actual Place ID
details = get_place_details(place_id)

if details:
    # Tidy the results and convert to DataFrame
    cleaned_df = tidy_nearby_result({"results": [details]})
    print(cleaned_df)
else:
    print("No details found for the given Place ID.")
